In [ ]:
# Advanced Deep Learning for Image Data using Transfer Learning (MobileNetV2)
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2, mobilenet_v2
from tensorflow.keras.utils import to_categorical

# --- Load small subset of CIFAR-10 ---
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train, y_train = x_train[:4000], y_train[:4000]   # smaller subset
x_test,  y_test  = x_test[:800],  y_test[:800]

# --- One-hot encode labels ---
y_train = to_categorical(y_train, 10)
y_test  = to_categorical(y_test, 10)

# --- Create tf.data pipelines with resize + preprocess ---
def preprocess(img, label):
    img = tf.image.resize(img, (96, 96))              # smaller image → less memory
    img = tf.cast(img, tf.float32)
    img = mobilenet_v2.preprocess_input(img)          # scale for MobileNetV2
    return img, label

train_ds = (tf.data.Dataset.from_tensor_slices((x_train, y_train))
            .shuffle(1000).map(preprocess).batch(16).prefetch(tf.data.AUTOTUNE))
val_ds = (tf.data.Dataset.from_tensor_slices((x_test, y_test))
          .map(preprocess).batch(16).prefetch(tf.data.AUTOTUNE))

# --- Build transfer-learning model ---
base = MobileNetV2(weights='imagenet', include_top=False, input_shape=(96, 96, 3))
base.trainable = False  # freeze pretrained layers

model = models.Sequential([
    base,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])

# --- Compile, train, and evaluate ---
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_ds, epochs=3, validation_data=val_ds)
loss, acc = model.evaluate(val_ds)
print(f"Validation Accuracy: {acc:.4f}")


Epoch 1/3
250/250 ━━━━━━━━━━━━━━━━━━━━ 14s 27ms/step - accuracy: 0.6691 - loss: 1.0357 - val_accuracy: 0.7887 - val_loss: 0.6263
Epoch 2/3
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.8728 - loss: 0.3567 - val_accuracy: 0.8200 - val_loss: 0.5089
Epoch 3/3
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9321 - loss: 0.2104 - val_accuracy: 0.8325 - val_loss: 0.5240
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8565 - loss: 0.4824
Validation Accuracy: 0.8325
